In [2]:
#import libraries
import xgboost as xg
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib as plt
from scipy.stats import norm
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from sklearn.metrics import r2_score
import data_ML as ML
import pickle

np.set_printoptions(suppress=True)

In [3]:
df = ML.getdata(False)


In [16]:
def modifydata(df):
    data = []
    df_calls = df[df.optionType == 1].drop(columns='optionType')
    df_american_calls = df_calls[df_calls.method == 1].drop(columns='method')
    # df_american_calls = df_american_calls.drop(columns='index')
    df_american_calls = df_american_calls[df_american_calls.lastPrice > 5]
    # df_american_calls = df_american_calls.drop(columns='index')
    y = np.asarray(df_american_calls[['lastPrice']])
    df_american_calls = df_american_calls.drop(columns='lastPrice')

    for line in range(len(df_american_calls)):
        data_tmp = df_american_calls.iloc[line]
        data.append(data_tmp)
    data = np.asarray(data)

    return data, y

X, y = modifydata(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1867, 5) (623, 5) (1867, 1) (623, 1)


In [17]:

# df_train, y_train, full_data2 = modifydata(df_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1867, 5) (623, 5) (1867, 1) (623, 1)


In [20]:
# Define the model
model_american_calls = Sequential()
model_american_calls.add(Dense(512, input_dim=5, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(256, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(256, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(64, activation='relu'))
model_american_calls.add(Dense(1))
model_american_calls.compile(optimizer='adam', loss='mse')

history = model_american_calls.fit(X_train, y_train, epochs=100, validation_split=0.2, batch_size=128)

# model_american_calls.save("model_american_calls.h5")
# model_american_calls.save("model_american_puts.h5")



Epoch 1/100
12/12 [==============================] - 0s 7ms/step - loss: 69530.7500 - val_loss: 83119.7031
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 62928.5508 - val_loss: 79999.4141
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 59534.2305 - val_loss: 82957.2891
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 58827.1016 - val_loss: 80276.6328
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 57451.2617 - val_loss: 79119.6875
Epoch 6/100
12/12 [==============================] - 0s 4ms/step - loss: 57074.8281 - val_loss: 77615.0078
Epoch 7/100
12/12 [==============================] - 0s 4ms/step - loss: 58538.9102 - val_loss: 78051.6016
Epoch 8/100
12/12 [==============================] - 0s 4ms/step - loss: 56872.7617 - val_loss: 78350.7500
Epoch 9/100
12/12 [==============================] - 0s 4ms/step - loss: 56918.0547 - val_loss: 81304.3281
Epoch 10/100
12/12 [=================

In [7]:
#prediction
import pandas as pd
y_pred = model_american_calls.predict(X_test)


db = pd.DataFrame({'prediction':y_pred.tolist(),'real':y_test.tolist()})
error = (y_pred - y_test)/y_test
db = db.assign(error = error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)
db.to_csv('./Data/MLPdata.csv')
db




20/20 [==============================] - 0s 562us/step
166.22925359681204


,prediction,real,error
0,[606.2442016601562],[56.15],9.796869
1,[562.3807983398438],[799.42],-0.296514
2,[201.25753784179688],[210.92],-0.045811
3,[211.6971435546875],[210.22],0.007027
4,[134.44834899902344],[139.0],-0.032746
...,...,...,...
618,[64.66755676269531],[64.39],0.004311
619,[74.28057861328125],[72.45],0.025267
620,[70.60993194580078],[311.75],-0.773505
621,[211.50531005859375],[219.9],-0.038175


In [8]:
y_test.shape

(623, 1)

In [9]:
DTR1 = DecisionTreeRegressor()
# cvs_DTR = cross_val_score(DTR, df_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
# DTR_score = cvs_DTR.mean()
# print(DTR_score)

DTR1.fit(X_train, y_train)
DTR1_pred = DTR1.predict(X_test)
DTR1_rmse = np.sqrt(mean_squared_error(y_test, DTR1_pred))
DTR1_rmse

60.81057192582081

In [10]:
y_pred = pd.DataFrame(DTR1_pred, columns=['prediction'])
final_test = pd.DataFrame(y_test, columns=['real'])

final_df = pd.concat((final_test, y_pred), axis=1)
final_df

,real,prediction
0,56.15,55.25
1,799.42,784.00
2,210.92,210.49
3,210.22,222.50
4,139.00,135.40
...,...,...
618,64.39,65.00
619,72.45,68.20
620,311.75,335.00
621,219.90,223.14


In [11]:

XGBr = xg.XGBRegressor(learning_rate=0.1, n_estimators=900)
cvs_XGBr = cross_val_score(XGBr, X_train, y_train, scoring='neg_root_mean_squared_error', cv=KFold(n_splits=10))
XGBr_score = cvs_XGBr.mean()
print(XGBr_score)


-28.400139537023126


In [12]:
XGBr.fit(X_train, y_train)
XGBr_pred = XGBr.predict(X_test)
XGBr_rmse = np.sqrt(mean_squared_error(y_test, XGBr_pred))
XGBr_rmse
saved_model = pickle.dumps(XGBr)

In [13]:
XGBr_saved_model = pickle.loads(saved_model)
def xgb_prediction(K, T, S_0, sigma , r):
    data = np.array([[K, T, S_0, sigma , r]])
    prediction = XGBr_saved_model.predict(data)
    return prediction



In [14]:
print(xgb_prediction(230, 246/365, 189.71, 0.3318, 0.269))


[24.331013]
